In [1]:
import pandas as pd
import numpy as np
import json
import os
from tabulate import tabulate

In [16]:
folder_path = r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\SUSY\SUSY QM\PennyLane\VQD\Destructive SWAP\SingleSWAP"
cutoffs = [2,4,8,16,32]
potentials = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]

In [17]:
converged_only = True
all_data = []

for potential in potentials:
    for cutoff in cutoffs:

        median = []
        mean = []
        min = []
        e0 = []
        e1 = []
        e2 = []
        ratios = []

        data_path = folder_path + "\\{}\\{}_{}.json".format(potential,potential, cutoff)
        
        with open(data_path, 'r') as file:
            data = json.load(file)

        num_VQD = data['num_VQD']

        converged = [all(x) for x in data['success']]
        converged_indices = [i for i, success in enumerate(converged) if success]
        converged_runs = sum(converged)

        if converged_only:
            results = [data['results'][i] for i in converged_indices]
        else:
            results = data['results']

        success_i = [i for i, success in enumerate(data['success']) if all(success)]
        successful_results = [data['results'][i] for i in success_i]
        energies =np.sort(successful_results)

        
        run_times = data['run_times']
        time_series = np.array([pd.Timedelta(t).total_seconds() for t in run_times])
        mean_vqe_time = np.mean(time_series)
        time_taken = np.mean(mean_vqe_time)

        for e in energies:
            
            e0.append(e[0])
            e1.append(e[1])
            e2.append(e[2])
            r = abs((e[2] - e[1]) / (e[2] - e[0]))
            ratios.append(r)

        e0_exact = data['exact_eigenvalues'][0]
        e1_exact = data['exact_eigenvalues'][1]
        e2_exact = data['exact_eigenvalues'][2]
        exact_ratio = abs((e2_exact - e1_exact) / (e2_exact - e0_exact))

        row = {
        "potential": potential,
        "cutoff": cutoff,
        'Converged Runs': f"{converged_runs}/100",
        'Mean VQD time (s)': f"{time_taken:.2f}",
        "e0": np.median(e0),
        "e1": np.median(e1),
        "e2": np.median(e2),
        "median_ratio": np.median(ratios),
        "mean_ratio": np.mean(ratios),
        #"min_ratio": np.min(ratios),
        "e0_exact": e0_exact,
        "e1_exact": e1_exact,
        "e2_exact": e2_exact,
        "exact_ratio": exact_ratio
        }

        all_data.append(row)

In [18]:
df = pd.DataFrame(all_data)
df

,potential,cutoff,Converged Runs,Mean VQD time (s),e0,e1,e2,median_ratio,mean_ratio,e0_exact,e1_exact,e2_exact,exact_ratio
0,AHO,2,100/100,96.26,-0.484619,-0.437500,1.472168,0.976087,0.977270,-0.437500,-0.437500,2.062500,1.000000e+00
1,AHO,4,100/100,110.61,-0.164785,0.631727,1.613018,0.551813,0.549880,-0.164785,0.673310,1.667943,5.427061e-01
2,AHO,8,100/100,152.23,0.032010,1.647396,1.795862,0.085613,0.088925,0.032010,1.680156,1.833526,8.513391e-02
3,AHO,16,97/100,296.42,-0.001167,1.652128,1.670107,0.017277,0.058781,-0.001167,1.677494,1.686381,5.266251e-03
4,AHO,32,94/100,942.05,0.000006,1.645029,1.674276,0.017542,0.057366,0.000006,1.686499,1.686559,3.585457e-05
5,DW,2,100/100,40.27,0.357233,0.733845,1.433856,0.650239,0.649631,0.357233,0.771447,1.478553,6.306019e-01
6,DW,4,100/100,66.61,0.902289,0.939845,1.661797,0.954120,0.945276,0.906560,0.950634,1.695666,9.441474e-01
7,DW,8,100/100,123.85,0.860071,0.888211,2.763990,0.983795,0.984296,0.884580,0.887725,2.693873,9.982617e-01
8,DW,16,100/100,172.59,0.867200,0.892520,2.785015,0.985690,0.985512,0.891599,0.891641,2.734122,9.999774e-01
9,DW,32,100/100,343.13,0.868212,0.891632,2.775115,0.986518,0.985509,0.891632,0.891632,2.734011,1.000000e+00


In [19]:
latex_table = tabulate(df.values, headers=df.columns, tablefmt="latex", floatfmt=".3f")
print(latex_table)

\begin{tabular}{lrlrrrrrrrrrr}
\hline
 potential   &   cutoff & Converged Runs   &   Mean VQD time (s) &     e0 &     e1 &    e2 &   median\_ratio &   mean\_ratio &   e0\_exact &   e1\_exact &   e2\_exact &   exact\_ratio \\
\hline
 AHO         &        2 & 100/100          &              96.260 & -0.485 & -0.437 & 1.472 &          0.976 &        0.977 &     -0.437 &     -0.437 &      2.062 &         1.000 \\
 AHO         &        4 & 100/100          &             110.610 & -0.165 &  0.632 & 1.613 &          0.552 &        0.550 &     -0.165 &      0.673 &      1.668 &         0.543 \\
 AHO         &        8 & 100/100          &             152.230 &  0.032 &  1.647 & 1.796 &          0.086 &        0.089 &      0.032 &      1.680 &      1.834 &         0.085 \\
 AHO         &       16 & 97/100           &             296.420 & -0.001 &  1.652 & 1.670 &          0.017 &        0.059 &     -0.001 &      1.677 &      1.686 &         0.005 \\
 AHO         &       32 & 94/100           &